In [52]:
import pandas as pd

In [54]:
training_df = pd.read_csv('nn.csv', index_col=0)
training_df.drop(columns=[training_df.columns[0], training_df.columns[2], training_df.columns[3], training_df.columns[len(training_df.columns)-1]], axis=1,  inplace=True)
training_df.dropna(inplace=True)
cols_at_end = ['Analyst']
training_df = training_df[[c for c in training_df if c not in cols_at_end] 
        + [c for c in cols_at_end if c in training_df]]

In [58]:
training_df

,Symbol,headline_polarity,convo_polarity,beta,profitMargins,forwardEps,bookValue,heldPercentInstitutions,shortRatio,shortPercentOfFloat,Analyst
1,AAP,0.25,0.21,1.30,0.06,12.76,52.44,1.03,6.92,0.07,Buy
5,ABC,0.19,0.27,0.53,-0.02,10.57,0.18,0.66,2.17,0.01,Buy
12,AEE,0.18,0.04,0.28,0.16,4.03,36.38,0.77,2.47,0.01,Buy
17,AFL,0.03,0.29,1.02,0.25,5.20,50.20,0.61,4.67,0.02,Buy
21,AIG,0.17,0.34,1.34,0.09,5.38,76.73,0.92,1.99,0.01,Buy
...,...,...,...,...,...,...,...,...,...,...,...
4298,ZTS,0.16,0.32,0.65,0.26,5.11,9.17,0.93,1.80,0.01,Strong Buy
4301,APRN,-0.02,0.18,-3.73,-0.13,-1.19,2.39,0.31,0.69,0.11,Strong Sell
4307,GCI,0.13,0.32,2.73,-0.09,0.46,3.86,0.64,11.92,0.13,Strong Sell
4312,NMM,0.44,0.35,2.57,0.55,15.14,48.00,0.06,0.27,0.00,Strong Sell
